In [1]:
from tqdm import tqdm
from collections import Counter
from itertools import combinations
from nltk.stem import PorterStemmer
from sklearn.externals import joblib 
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.metrics.pairwise import euclidean_distances

import imp
import copy
import pickle
import multiprocessing

import numpy as np
import pandas as pd
import utils as my_utils
import ELJST_script_unigram as lda
import matplotlib.pyplot as plt

/home/william18026/.conda/envs/python3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
min_df = 5
max_df = .5
maxIters = 5

beta = .01
gamma = 10
n_topics = 5
n_sentiment = 5
lambda_param = 1.0

In [3]:
cutoff = 0.3
dataset_name = "amazon_electronics"
n_docs = 100000

In [4]:
dataset = pd.read_pickle("resources/"+ dataset_name + "_" + str(n_docs) + "_dataset")

In [10]:
embedding_name = "glove_0.6"

In [11]:
similar_words = pickle.load(open("resources/"+ dataset_name + "_" + str(n_docs) + "_" +embedding_name + ".pickle","rb"))

In [12]:
alpha = 0.1/n_topics * np.ones(n_topics)
gamma = [gamma/(n_topics*n_sentiment)]*n_sentiment

In [ ]:
# imp.reload(lda)

In [25]:
sampler = lda.SentimentLDAGibbsSampler(n_topics, alpha, beta, gamma, numSentiments=n_sentiment, minlabel = 0, 
                                       maxlabel = 5, SentimentRange = 5, max_df = max_df, min_df = min_df, 
                                       lambda_param = lambda_param)

In [ ]:
%%time
sampler._initialize_(reviews = dataset.text.tolist(), labels = dataset.sentiment.tolist())

In [ ]:
sampler.wordOccuranceMatrix.sum()

In [24]:
sampler.run(reviews=dataset.text.tolist(), labels=dataset.sentiment.tolist(), 
            similar_words=similar_words, mrf=True, maxIters=maxIters)

  0%|          | 1/100000 [00:00<4:57:31,  5.60it/s]

Starting iteration 1 of 5


 14%|█▍        | 13783/100000 [40:44<6:40:31,  3.59it/s] 

KeyboardInterrupt: 

In [22]:
Counter([len(i) for i in sampler.docs_edges])

Counter()

# Evaluations

In [ ]:
plt.plot(sampler.loglikelihood_history)

In [ ]:
silhouette_score(euclidean_distances(sampler.wordOccuranceMatrix),
                 sampler.dt_distribution.argmax(axis=1), metric='precomputed')

In [ ]:
davies_bouldin_score(sampler.wordOccuranceMatrix, sampler.dt_distribution.argmax(axis=1))

In [ ]:
my_utils.coherence_score(sampler.wordOccuranceMatrix, list(sampler.getTopKWords(5).values()), sampler.vocabulary)

In [ ]:
%%time
my_utils.get_hscore_multi(sampler.dt_distribution, sampler.wordOccuranceMatrix, n_topics, 500)

In [ ]:
sampler.loglikelihood()

In [ ]:
sampler.perplexity()

### Appendix

In [ ]:
# p = [item for sublist in dataset['cleaned'].tolist() for item in sublist]

In [ ]:
# sorted(Counter(p))

In [ ]:
# def process_l(s):
#     return [i.lemma_ for i in sp(s) if i.lemma_ not in '-PRON-']

In [ ]:
# l = dataset['text'].tolist()

In [ ]:
# pool = multiprocessing.Pool(n_cores)
# processed_l = pool.map(process_l, l)
# pool.close()

In [ ]:
# joblib.dump(sampler, "resources/sampler_20iter_0.5_1")

In [ ]:
# pickle_out = open("resources/amazon_muiscal_glove_0.4.pickle","wb")
# pickle.dump(similar_words, pickle_out)
# pickle_out.close()